In [34]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

df=pd.read_csv("telecom_customer_churn.csv")
df.shape

(6589, 38)

In [37]:
columns_to_drop=['Customer ID','City','Zip Code','Latitude','Longitude','Churn Category','Churn Reason','Offer','Gender','Married','Payment Method','Internet Type']
data=df.drop(columns=columns_to_drop)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6589 entries, 0 to 6588
Data columns (total 26 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Age                                6589 non-null   int64  
 1   Number of Dependents               6589 non-null   int64  
 2   Number of Referrals                6589 non-null   int64  
 3   Tenure in Months                   6589 non-null   int64  
 4   Phone Service                      6589 non-null   object 
 5   Avg Monthly Long Distance Charges  5945 non-null   float64
 6   Multiple Lines                     5945 non-null   object 
 7   Internet Service                   6589 non-null   object 
 8   Avg Monthly GB Download            5245 non-null   float64
 9   Online Security                    5245 non-null   object 
 10  Online Backup                      5245 non-null   object 
 11  Device Protection Plan             5245 non-null   objec

In [31]:
data1=data.copy()
cols_to_no=['Streaming TV','Streaming Movies','Streaming Music','Unlimited Data','Multiple Lines','Device Protection Plan','Premium Tech Support','Online Security','Online Backup']
data1['Avg Monthly Long Distance Charges'].fillna(0,inplace=True)
data1['Avg Monthly GB Download'].fillna(0,inplace=True)
data1[cols_to_no]=data1[cols_to_no].fillna("No")
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6589 entries, 0 to 6588
Data columns (total 26 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Age                                6589 non-null   int64  
 1   Number of Dependents               6589 non-null   int64  
 2   Number of Referrals                6589 non-null   int64  
 3   Tenure in Months                   6589 non-null   int64  
 4   Phone Service                      6589 non-null   object 
 5   Avg Monthly Long Distance Charges  6589 non-null   float64
 6   Multiple Lines                     6589 non-null   object 
 7   Internet Service                   6589 non-null   object 
 8   Avg Monthly GB Download            6589 non-null   float64
 9   Online Security                    6589 non-null   object 
 10  Online Backup                      6589 non-null   object 
 11  Device Protection Plan             6589 non-null   objec

In [20]:
data2=data1.copy()
cols_to_scale = ['Age','Number of Dependents','Number of Referrals','Tenure in Months','Avg Monthly Long Distance Charges','Avg Monthly GB Download','Monthly Charge', 'Total Charges',
       'Total Extra Data Charges', 'Total Long Distance Charges','Total Revenue']

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data2[cols_to_scale] = scaler.fit_transform(data2[cols_to_scale])
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6589 entries, 0 to 6588
Data columns (total 26 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Age                                6589 non-null   float64
 1   Number of Dependents               6589 non-null   float64
 2   Number of Referrals                6589 non-null   float64
 3   Tenure in Months                   6589 non-null   float64
 4   Phone Service                      6589 non-null   object 
 5   Avg Monthly Long Distance Charges  6589 non-null   float64
 6   Multiple Lines                     6589 non-null   object 
 7   Internet Service                   6589 non-null   object 
 8   Avg Monthly GB Download            6589 non-null   float64
 9   Online Security                    6589 non-null   object 
 10  Online Backup                      6589 non-null   object 
 11  Device Protection Plan             6589 non-null   objec

In [36]:
data2=data.copy()
numerical_columns = data2.select_dtypes(include=['float64', 'int64']).columns
for col in numerical_columns:
    
    Q1 = data2[col].quantile(0.25)
    Q3 = data2[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    data2[col][(data2[col] < lower_bound) | (data2[col] > upper_bound)] = data2[col].median()
data2.shape

C:\Users\harii\AppData\Local\Temp\ipykernel_20680\3395071953.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2[col][(data2[col] < lower_bound) | (data2[col] > upper_bound)] = data2[col].median()


(6589, 26)

In [22]:
categorical_columns = data2.select_dtypes(include=['object']).columns
data3=data2.copy()
label_encoder = LabelEncoder()

for col in categorical_columns:
    data3[col] = label_encoder.fit_transform(data3[col])
    
data3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6589 entries, 0 to 6588
Data columns (total 26 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Age                                6589 non-null   float64
 1   Number of Dependents               6589 non-null   float64
 2   Number of Referrals                6589 non-null   float64
 3   Tenure in Months                   6589 non-null   float64
 4   Phone Service                      6589 non-null   int32  
 5   Avg Monthly Long Distance Charges  6589 non-null   float64
 6   Multiple Lines                     6589 non-null   int32  
 7   Internet Service                   6589 non-null   int32  
 8   Avg Monthly GB Download            6589 non-null   float64
 9   Online Security                    6589 non-null   int32  
 10  Online Backup                      6589 non-null   int32  
 11  Device Protection Plan             6589 non-null   int32

In [26]:
correlation_matrix = data3.corr()
highly_correlated_vars = set()
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.6:
            colname = correlation_matrix.columns[i]
            highly_correlated_vars.add(colname)

data3= data3.drop(columns=highly_correlated_vars)
data3.info()
highly_correlated_vars

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6589 entries, 0 to 6588
Data columns (total 19 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Age                                6589 non-null   float64
 1   Number of Dependents               6589 non-null   float64
 2   Number of Referrals                6589 non-null   float64
 3   Tenure in Months                   6589 non-null   float64
 4   Phone Service                      6589 non-null   int32  
 5   Avg Monthly Long Distance Charges  6589 non-null   float64
 6   Multiple Lines                     6589 non-null   int32  
 7   Internet Service                   6589 non-null   int32  
 8   Avg Monthly GB Download            6589 non-null   float64
 9   Online Security                    6589 non-null   int32  
 10  Online Backup                      6589 non-null   int32  
 11  Device Protection Plan             6589 non-null   int32

set()

In [27]:
from statsmodels.stats.outliers_influence import \
variance_inflation_factor


vif = pd.DataFrame()
vif['variable']= data3.columns

vif['vif'] = [variance_inflation_factor(data3.values,i)
 for i in range(data3.shape[1])]

vif

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1738: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.uncentered_tss


,variable,vif
0,Age,4.180635
1,Number of Dependents,NaN
2,Number of Referrals,1.442046
3,Tenure in Months,5.318455
4,Phone Service,9.555916
5,Avg Monthly Long Distance Charges,4.232830
6,Multiple Lines,2.331786
7,Internet Service,11.253685
8,Avg Monthly GB Download,3.970828
9,Online Security,1.939655


In [9]:
data3.drop(columns=['Total Extra Data Charges','Total Refunds',''],inplace=True)

In [10]:
x = data3.iloc[:,:-1].values
y = data3.iloc[:,-1].values

x=pd.DataFrame(x)
dependant_col='Customer Status'
cols=[col for col in data3.columns if col != dependant_col]
x.columns=cols

y

array([1, 1, 0, ..., 0, 1, 1], dtype=int64)

In [11]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,
 test_size=0.3, random_state=0)
x_train

,Age,Number of Dependents,Number of Referrals,Tenure in Months,Phone Service,Avg Monthly Long Distance Charges,Multiple Lines,Internet Service,Avg Monthly GB Download,Online Security,Online Backup,Device Protection Plan,Premium Tech Support,Streaming TV,Streaming Movies,Paperless Billing,Total Refunds,Total Extra Data Charges
2267,0.344262,0.000000,0.181818,0.042254,1.0,0.569514,0.0,1.0,0.058824,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.066667
3538,0.770492,0.000000,0.000000,1.000000,1.0,0.047409,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.00,0.000000
2616,0.819672,0.000000,0.000000,0.774648,1.0,0.933587,1.0,1.0,0.211765,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.00,0.000000
5420,0.770492,0.000000,0.000000,0.690141,1.0,0.212042,1.0,1.0,0.164706,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.00,0.000000
5657,0.016393,0.333333,0.272727,0.929577,1.0,0.408282,1.0,1.0,0.611765,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.00,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4931,0.065574,0.333333,0.181818,0.633803,1.0,0.125225,0.0,1.0,0.564706,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.00,0.333333
3264,0.688525,0.000000,0.090909,0.084507,1.0,0.102621,0.0,1.0,0.047059,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.00,0.000000
1653,0.065574,0.000000,0.090909,0.507042,0.0,0.000000,0.0,1.0,0.317647,1.0,0.0,0.0,1.0,1.0,0.0,0.0,44.13,0.000000
2607,0.950820,0.111111,0.454545,0.985915,1.0,0.642529,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000


In [12]:
from sklearn.linear_model import LogisticRegression
reg = LogisticRegression(solver='lbfgs', max_iter=5000)
reg.fit(x_train,y_train)

LogisticRegression(max_iter=5000)

In [13]:
y_pred = reg.predict(x_test)

In [14]:
from sklearn.metrics import r2_score, mean_squared_error,accuracy_score
r2_score(y_test,y_pred)


0.051006258000284355

In [15]:
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.8088012139605463